# Projekt NLP - Beschreibung und Exploration 

Ziel des Projekts ist das Analysieren und Clustern der Arikel des Heise Online Journals in der Rubrik Wirtschaft.

Gruppenmitglieder: 

    - Christian Gunzelmann
    - Diane Mittermair 

# Projektbeschreibung

Dieses Projekt wurde im Zuge des Kurses Natural Language Processing durchgeführt.
Ziel des Projekts ist die Analyse der Daten des Heise Online Journals, speziell das Clustern der Artikel, welche in der Rubrik Wirtschaft veröffentlicht wurden.

https://www.heise.de/newsticker/wirtschaft/

Die Daten wurden mit dem im Notebook ''Download'' aufgeführten Code heruntergeladen. Es folgt die Analyse der Daten in der Exploration. Hier wurden die Daten für einen besseren Überblick zuerst im Einzelnen analysiert. Um ein optimales Ergebniss in der Clusteranalyse zu erhalten, wurde im Abschnitt 6 speziell auf das Feature verwandte Themen (related topics) eingegangen. Das Clustering erfolgt letztendlich in einem zusätzlichen Notebook ''Clustering''.


In [1]:
import json
from matplotlib_venn import venn2
import plotly.express as px
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import re
import pandas as pd
import itertools
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import spacy
import nltk
from spacy.lang.de.stop_words import STOP_WORDS
nlp = spacy.load("de_core_news_sm")

In [2]:
with open("articles.json", "r") as f:
    articles = json.load(f)

In [3]:
articles

[{'article_url': 'https://www.heise.de/news/Telekom-und-Apple-starten-Augmented-Reality-Innovationsprogramm-6147126.html',
  'number_comments': 0,
  'date': '07-26-2021, 16:01',
  'brandig': ['Mac & i'],
  'related_topics': ['5G',
   'Apple',
   'Augmented Reality',
   'Deutsche Telekom',
   'Innovation',
   'iOS',
   'iPhone',
   'Mobilfunk',
   'Startups',
   'Wettbewerb'],
  'authors': ['Ben Schwan'],
  'heading': 'Telekom und Apple starten Augmented-Reality-Innovationsprogramm',
  'lead': 'Im Rahmen des Start-up-Inkubators "Hubraum" können sich Entwickler in sechs Kategorien bewerben, um Apples AR-Technik mit 5G und Co. zu kombinieren.',
  'atricle_text': 'Die Deutsche Telekom AG hat zusammen mit ihrer US-Tochter T-Mobile sowie Apple ein Innovationsprogramm für Augmented Reality (AR) aufgelegt. Die Kampagne läuft im Rahmen des Telekom-Inkubators Hubraum mit Sitz in Deutschland und ist international angelegt. Bewerbungen sind noch bis zum 4. August möglich, danach wird es für die in

# Aufbau des Notebooks

Dieses Notebook dient als grundsätzliche Dataexploration. Die einzelnen Merkmale eines Artikels werden statistisch analysiert und graphisch aufbereitet. Zu Beginn jedes Merkmals werden grundsätzliche Erkennisse textuell erläutert, im weiteren Verlauf befindet sich der Code, Tabellen und Grafiken. Das Notebook ist wie folgt strukturiert:

- 3 Kommentare
- 4 Datum
- 5 Branding/Tag
- 6 Verwandte Themen
- 7 Autoren
- 8 Überschrift
- 9 Lead
- 10 Text

# Kommentare
Im Schnitt umfasst ein Artikel 134 Kommentare. Die meist diskutierten Artikel haben sogar über 3000 Kommentare, wobei sich die Top drei alle auf das Thema "Tempolimit" fokussieren und generell scheint das Thema "Auto" eine sehr große Rolle zu spielen.

In [ ]:
number_comments_list = [article["number_comments"] for article in articles]
number_comments_np = np.array(number_comments_list)

## Verteilung

In [ ]:
f = plt.figure(figsize=(20,5))
plt.subplot(1, 2, 1)
plt.hist(number_comments_np)
plt.xlabel('Anzahl Kommentare')
plt.ylabel('Häufigkeit')
plt.title('Kommentar Histogram')
plt.subplot(1, 2, 2)
plt.hist(np.sort(number_comments_np)[:12000])
plt.xlabel('Anzahl Kommentare')
plt.ylabel('Häufigkeit')
plt.title('Kommentar Histogram (Zoomed)')
plt.show(f)

In [ ]:
np.mean(number_comments_np)

In [ ]:
f = plt.figure(figsize=(20,5))
plt.subplot(1, 2, 1)
plt.boxplot(number_comments_np)
plt.title('Boxplot Kommentare')
plt.subplot(1, 2, 2)
plt.boxplot(number_comments_np,showfliers=False)
plt.title('Boxplot Kommentare ohne Outlier')
plt.show(f)

## Meist diskutierte Artikel 🚗

In [ ]:
for idx in reversed(np.argsort(number_comments_np)[-10:]):
    print(f"Anzahl Kommentare {articles[idx]['number_comments']}")
    print(f"Überschrift \"{articles[idx]['heading']}\"")
    print(f"Datum {articles[idx]['date']}")
    print()

### Artikel ohne Kommentare

In [ ]:
np.count_nonzero(number_comments_np==0)

# Datum
<b>Veröffenlichung nach Jahren</b><br>
Seit 2003 wurden in der Rubrik Wirtschaft Artikel veröffentlicht, allerdings erst ab 2017 mit erhöhter Häufigkeit (> 2000). Seit 2017 steigt die Anzahl der publizierten Artikel (2021 bei bereits der Hälfte des Jahres mehr Artikel wie im Jahre 2017).<br>
<b>Veröffenlichung nach Monaten</b><br>
Im Juli werden am meisten Artikel veröffenlticht (ca. 200 mehr als der Durchschnitt).<br>
Im Februar werden am wenigsten Artikel veröffenlticht (ca. 170 weniger als der Durchschnitt).<br>
<b>Veröffenlichung nach Stunden</b><br>
Es werden rund um die Uhr Artikel veröffentlicht, für gewöhnlich aber zwischen 6 und 19 Uhr. Um 10 Uhr werden am meisten Artikel veröffenlicht.

In [ ]:
number_comments_list = [article["number_comments"] for article in articles]
number_comments_np = np.array(number_comments_list)

## Artikel pro Jar

In [ ]:
articles_year = {}
for article in articles:
    date = article["date"]
    date = datetime.strptime(date, "%m-%d-%Y, %H:%M")
    articles_year[date.year] = articles_year.get(date.year,0) + 1

In [ ]:
plt.figure(figsize=(20,5))
plt.bar(range(len(articles_year)), list(articles_year.values()), align='center')
plt.xticks(range(len(articles_year)), list(articles_year.keys()))
plt.xlabel('Jahr')
plt.ylabel('Häufigkeit')
plt.gca().invert_xaxis()
plt.show()

In [ ]:
articles_year

## Artikel nach Monat (ohne 2021)

In [ ]:
articles_mo = {}
for article in articles:
    date = article["date"]
    date = datetime.strptime(date, "%m-%d-%Y, %H:%M")
    if date.year != 2021:
        articles_mo[date.strftime("%B")] = articles_mo.get(date.strftime("%B"),0) + 1

In [ ]:
plt.figure(figsize=(20,5))
plt.bar(range(len(articles_mo)), list(articles_mo.values()), align='center')
plt.xticks(range(len(articles_mo)), list(articles_mo.keys()))
plt.xlabel('Monat')
plt.ylabel('Häufigkeit')
plt.gca().invert_xaxis()
plt.show()

In [ ]:
articles_mo["July"] - np.mean(list(articles_mo.values()))

In [ ]:
np.mean(list(articles_mo.values())) - articles_mo["February"]

## Artikel nach Stunde (ohne 2021)

In [ ]:
articles_hour = {}
for article in articles:
    date = article["date"]
    date = datetime.strptime(date, "%m-%d-%Y, %H:%M")
    if date.year != 2021:
        articles_hour[date.hour] = articles_hour.get(date.hour,0) + 1
articles_hour = {key: value for (key, value) in sorted(articles_hour.items(), key=lambda x: x[0], reverse=True)}

In [ ]:
plt.figure(figsize=(20,5))
plt.bar(range(len(articles_hour)), list(articles_hour.values()), align='center')
plt.xticks(range(len(articles_hour)), list(articles_hour.keys()))
plt.xlabel('Stunde')
plt.ylabel('Häufigkeit')
plt.gca().invert_xaxis()
plt.show()

# Branding/Tag

Es gibt insgesamt 10 verschiedene Tags. Ein Tag gibt das jeweilige Heise-Magazin an, in dem der Artikel erschienen ist. Folglich hat ein Artikel entweder ein oder gar kein Tag. Nur ca. 17% (2715) aller Artikel besitzen ein Tag. Die Tags "Mac & i" und "Autos" treten mit 1012 am häufigsten auf, während das Tag "Telepolis" nur einmal vorkommt.

In [ ]:
tags = [tag for article in articles for tag in article["brandig"]]

In [ ]:
print(len(set(tags)))
set(tags)

In [ ]:
print(f"Anzahl Artikel ohne Tags: {len(articles) - len(tags)}")
print(f"Anzahl Artikel mit Tags: {len(tags)}")
print(f"Anteil Artikel mit Tags: {len(tags)/(len(articles) + len(tags))}")

In [ ]:
tag_frequencies = {}
for tag in tags:
    tag_frequencies[tag] = tag_frequencies.get(tag,0)+1
tag_frequencies = {key: value for (key, value) in sorted(tag_frequencies.items(), key=lambda x: x[1], reverse=True)}

In [ ]:
tag_frequencies

In [ ]:
plt.figure(figsize=(20,5))
plt.bar(range(len(tag_frequencies)), list(tag_frequencies.values()), align='center')
plt.xticks(range(len(tag_frequencies)), list(tag_frequencies.keys()))
plt.xlabel('Tag')
plt.ylabel('Häufigkeit')
plt.gca().invert_xaxis()
plt.show()

# Verwandte Themen
Ein Artikel kann mehrere verwandte Themen oder auch gar keine haben. Bei ca. 7,5 % fehlt der Eintrag. Es gibt insgesamt 913 verschiedene verwandte Themen, wobei "Elektromobilität", "Elektroauto" und "Apple" die häufigsten Themen sind. Dies spiegelt auch die Analyse im vorherigen Abschnitt über die Tags wieder (häufigste Tags "Mac & i" und "Autos"). Außerdem fällt auf, dass das Coronavirus extrem häufig thematisiert wurde, so ist es bereits das fünft häufigste Thema, und das obwohl es erst Beginn 2020 relevant wurde.<br><br>
<b>Themen-Abdeckung</b><br>
Um einen ersten Ansatzpunkt für eine gute Anzahl an Clustern beim folgenden Clustering zu haben wird zuerst geschaut, wie viele Artikel bereits ein verwandtes Thema aus den Top X Themen haben. So macht es beispielsweise keinen Sinn 
eine Clusteranalyse mit 800 Clustern durchzuführen, wenn bereits 90% der Artikel mindestens eines der Top 100 Themen in ihren verwandten Themen beinhalten. 
<br> Das <b>Ergebnis</b> zeigt, dass 90% der Artikel mindestens eines der Top 265 Themen als verwandtes Thema enthalten und
80% der Artikel mindestens eines der Top 75 Themen enthalten. Da bei nur insgesamt 92,5% der Artikel verwandte Themen angegeben sind, scheint ein Clustering von über 300 Clustern kaum sinvoll.<br><br>
<b>Themen die häufig zusammen auftreten</b><br>
Häufig zusammen auftretende Themen sind möglicherweise inhaltlich sehr ähnlich und folglich durch Clustering weniger gut zu unterscheiden bzw. gar nicht, wenn es sich um Unterthemen handelt.<br>
<b>Ergebnis</b>:
Betrachtet man bereits die häufigsten drei Themen-Paare stellt man fest, dass Elektroauto, Facebook und iPhone genau so häufig im Paar (mit Elektromobilität, Social Media und Apple) auftreten, wie sie generell auftreten. Diese Themen sind folglich als Unterthemen zu betrachten. Insgesamt gibt es 394 solcher hundertprozentiger Unterthemen (siehe Venn-Diagramme) und 536 Themen, die bei mehr als der Hälfte ihres Auftretens als Paar mit einem bestimmten Thema vorkommen. Folglich sind über die Hälfte der Themen kaum bzw. nicht von anderen Themen unterscheidbar, da es sich um Unterthemen handelt.<br><br>
<b>Veränderung der verwanten Theman nach Jahren</b><br>
Es zeigt sich, dass die Themen Elektromobilität, Elektoauto, Apple und Geschäftszahlen (im Jahr 2021 auf Platz 7) in allen Jahren eine sehr große Bedeutung gespielt haben. Zusätzlich gibt es weitere folgende Themen, welche jeweils in den verschiedenen Jahren eine Rolle gespielt haben:

    - 2017 EU
    - 2018 Abgas-Skandal
    - 2019 Mobilfunk
    - 2020 Coronavirus
    - 2021 Coronavirus, Klimawandel
    
<b>Meist diskutierte Artikel</b><br>
"Elektromobilität" und "Elektroauto" sind auch hier auf Platz eins und die Themen mit den meisten Kommentaren. Auffällig ist, dass das Thema <b>"Tesla Motors"</b> sehr häufig diskutiert wurde (Platz 3), obwohl es nicht einmal zu den 20 häufigsten Themen gehört. Dies verdeutlicht die Relevanz dieses Themas für die Leserinnen und Leser. Das Thema Geschäftszahlen scheint hingegen wenig zu interessieren, da es trotz seines hohen Auftretens nicht einmal zu den 20 am häufigsten diskutierten Themen zählt.

## Welche Themen gibt es?

In [ ]:
related_topics_per_article = [article["related_topics"] for article in articles]

In [ ]:
related_topics_freq = {}
for topics in related_topics_per_article:
    for topic in topics:
        related_topics_freq[topic] = related_topics_freq.get(topic, 0) + 1

In [ ]:
counter=0
# Wie viele leere Elemente sind in der Liste? 
for related_topics in related_topics_per_article:
    if not related_topics:
        counter += 1
print('Anzahl der Einträge insgesammt:',len(related_topics_freq) ,'Anzahl der leeren Einträge:',counter)   

In [ ]:
wordcloud = WordCloud(background_color="white",width=1920, height=1080, max_words=200).generate(" ".join([related_topic for related_topics in related_topics_per_article for related_topic in  related_topics]))

plt.figure(figsize=(12, 7), dpi=80)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
topics_df = pd.DataFrame(related_topics_freq.items(),columns=['topic_name','frequency'])
topics_df = topics_df.sort_values(by='frequency',ascending=False)
topics_df["rel_frequency_article"] = topics_df["frequency"]/len(articles)
topics_df.head(20)

In [ ]:
fig = px.bar(topics_df.head(20), x= 'topic_name', y = 'frequency')
fig.show()

## Wie viele Artikel werden von den Top X Themen abgedeckt?

In [ ]:
topic_coverage = []
for i in range(914):
    top_topics = list(topics_df.head(i)["topic_name"])
    counter = 0
    for related_topics in  related_topics_per_article:
        for related_topic in related_topics:
            if related_topic in top_topics:
                counter +=1
                break
    topic_coverage.append(counter/13631)

In [ ]:
fig = px.area(
    x=range(len(topic_coverage)),
    y=topic_coverage,
    labels={"x": "Top Themen", "y": "Anteil der Artikel die eines der Top Themen enthalten"}
)
fig.show()

## Welche Themen treten häufig zusammen auf?

In [ ]:
topic_pairs = []
for topics in related_topics_per_article:
    topic_pairs += list(itertools.combinations(topics, 2))

In [ ]:
topic_pairs_df = pd.DataFrame(np.ones(len(topic_pairs)))
topic_pairs_df["topic_pair"] = topic_pairs

In [ ]:
topic_pairs_df = topic_pairs_df.groupby(["topic_pair"]).count()
topic_pairs_df = topic_pairs_df.reset_index()
topic_pairs_df = topic_pairs_df.rename(columns={0:"number_topic_pair"})
topic_pairs_df = topic_pairs_df.sort_values(by='number_topic_pair',ascending=False)
topic_pairs_df.head(10)

In [ ]:
topic_articles = {}

for i, related_topics in  enumerate(related_topics_per_article): 
    for topic in related_topics:
        topic_articles[topic] = topic_articles.get(topic,[]) + [i]  

In [ ]:
subtopics = set({})
for topic1, topic2 in set(topic_pairs):
    topic1_articles = set(topic_articles[topic1])
    topic2_articles = set(topic_articles[topic2])
    intersection_set = topic1_articles.intersection(topic2_articles)
    
    if intersection_set == topic1_articles:
        venn2([topic1_articles,topic2_articles], set_labels = [topic1, topic2])
        plt.show()
        subtopics.add(topic1)
        
    elif intersection_set == topic2_articles:
        venn2([topic2_articles,topic1_articles], set_labels = [topic2, topic1])
        plt.show()
        subtopics.add(topic2)

In [ ]:
len(subtopics)

In [ ]:
subtopics_50 = set({})
for topic1, topic2 in set(topic_pairs):
    topic1_articles = set(topic_articles[topic1])
    topic2_articles = set(topic_articles[topic2])
    intersection_set = len(topic1_articles.intersection(topic2_articles))
    
    if intersection_set / len(topic1_articles) > 0.5:
        subtopics_50.add(topic1)
        
    elif intersection_set / len(topic2_articles) > 0.5:
        subtopics_50.add(topic2)

In [ ]:
len(subtopics_50)

## Wie haben sich die Themen im Laufe der Jahre geändert?

In [ ]:
year_topic = [[datetime.strptime(article["date"], "%m-%d-%Y, %H:%M").year, topic] for article in articles for topic in article["related_topics"]]
year_topic = pd.DataFrame(year_topic, columns=["year","topic"])

Verwndung von Plotly, um durch die Interaktivität schneller sehen zu können, welche Farbe welches Thema hat.

In [ ]:
year_topic_freq_df = year_topic.groupby(['year','topic']).size().groupby('year', group_keys=False).nlargest(5).reset_index(name="frequency")
# Nur die Jahre 2017 bis 2021
year_topic_freq_df = year_topic_freq_df[year_topic_freq_df.year.isin(range(2017,2022))]
fig = px.bar(year_topic_freq_df, x="year", y="frequency", color="topic", title="Anzahl Artikel Veröffenlichung nach Themen Top 5")
fig.show()
# year_topic_freq_pivot = year_topic_freq_df.pivot(index="year",columns="topic", values="frequency")
# #Sortieren nach NaN Werten, um besseren Barplot zu erhalten
# year_topic_freq_pivot = year_topic_freq_pivot[year_topic_freq_pivot.isnull().sum().sort_values(ascending=True).index]

# fig, ax = plt.subplots(figsize=(20,10))
# year_topic_freq_pivot.plot.bar(stacked=True, ax=ax)
# plt.show()

In [ ]:
year_topic_freq_df = year_topic.groupby(['year','topic']).size().groupby('year', group_keys=False).nlargest(10).reset_index(name="frequency")
# Nur die Jahre 2017 bis 2021
year_topic_freq_df = year_topic_freq_df[year_topic_freq_df.year.isin(range(2017,2022))]
fig = px.bar(year_topic_freq_df, x="year", y="frequency", color="topic", title="Anzahl Artikel Veröffenlichung nach Themen Top 10")
fig.show()

## Welche Themen werden am meisten diskutiert?

In [ ]:
topic_ncomments = {}
wordcloud_text = [] # Um eine Wordcloud auf Basis der Anzahl der Kommentare erstellen zu können
for article in articles:
    for topic in article["related_topics"]:
        topic_ncomments[topic] = int(article["number_comments"]) + topic_ncomments.get(topic,0)

In [ ]:
topic_ncomments_df = pd.DataFrame(topic_ncomments.items(),columns=['topic_name','number of comments'])
topic_ncomments_df = topic_ncomments_df.sort_values(by='number of comments',ascending=False)

In [ ]:
wordcloud = WordCloud(background_color="white",width=1920, height=1080, max_words=200).generate_from_frequencies(topic_ncomments)
plt.figure(figsize=(12, 7), dpi=80)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()
print("Je größer die Schrift desto häufiger diskutiert")

In [ ]:
fig = px.bar(topic_ncomments_df.head(20), x= 'topic_name', y = 'number of comments')
fig.show()
print("Vergleich zu Häufigkeit der Themen")
fig = px.bar(topics_df.head(20), x= 'topic_name', y = 'frequency')
fig.show()

## Wie änderte sich das Interesse der Leser im Verlauf der Jahre?

In [ ]:
year_topic_comments = [[datetime.strptime(article["date"], "%m-%d-%Y, %H:%M").year, topic, article["number_comments"]] for article in articles for topic in article["related_topics"]]
year_topic_comments = pd.DataFrame(year_topic_comments, columns=["year","topic","number_comments"])
#nur 2017 - 2021
year_topic_comments = year_topic_comments[year_topic_comments.year.isin(range(2017,2022))]

In [ ]:
year_topic_comments = year_topic_comments.groupby(['year','topic']).sum().reset_index()
top_5_year_topic_comments = year_topic_comments.groupby('year').apply(lambda x: x.nlargest(5, "number_comments")).reset_index(drop=True)
top_10_year_topic_comments = year_topic_comments.groupby('year').apply(lambda x: x.nlargest(10, "number_comments")).reset_index(drop=True)

In [ ]:
fig = px.bar(top_5_year_topic_comments, x="year", y="number_comments", color="topic", title="Anzahl an Kommentaren Top 5")
fig.show()
print("Vergleich zur Anzahl der veröffentlichten Artikel")
year_topic_freq_df = year_topic.groupby(['year','topic']).size().groupby('year', group_keys=False).nlargest(5).reset_index(name="frequency")
# Nur die Jahre 2017 bis 2021
year_topic_freq_df = year_topic_freq_df[year_topic_freq_df.year.isin(range(2017,2022))]
fig = px.bar(year_topic_freq_df, x="year", y="frequency", color="topic", title="Anzahl Artikel Veröffenlichung nach Themen Top 5")
fig.show()
fig = px.bar(top_10_year_topic_comments, x="year", y="number_comments", color="topic", title="Anzahl an Kommentaren Top 10")
fig.show()

# Autoren

In den meisten Fällen stammt ein Artikel von nur einem Autor. Neben der dpa (4665) sind Andreas Wilkens(1556), Stefan Krempl(845) und Ben Schwan(712)	die Autoren, die am meisten Artikel geschrieben haben. Wobei Andreas Wilkens (Elektro-)Autos, Stefan Krempl EU und Datenschutz und Ben Schwan Apple thematisiert. Autorenpaare treten relativ selten auf. Das Autorenpaar mit den meisten Artikel (ohne dpa also zweit Autor) sind Angela Froitzheim und Ulf J. Froitzheim mit lediglich 24 Artikeln 

## clean
Manchmal durch "und" oder "," und nicht als neues Listenelement

In [ ]:
authors_article = [article["authors"] for article in articles]

In [ ]:
for i, authors in enumerate(authors_article):
    tmp_elem = []
    for author_s in authors:
        tmp_elem += re.split(',| und |;|/', author_s)
        
    authors_article[i] = [author.strip() for author in tmp_elem]

In [ ]:
for authors in authors_article:
    for i, author in enumerate(authors):
        if '' == author:
            authors.remove(author)
        if '-Korrespondenten' == author:
            authors.remove(author)
        if '. Felix Frieler' == author:
            authors[i] = 'Felix Frieler'
        if 'Grundhoff' == author:
            authors[i] = 'Stefan Grundhoff'
        if 'Martin Fischer (translation by Fabian A. Scherschel)' == author:
            authors[i] = 'Martin Fischer'
            authors.append("Fabian A. Scherschel")
        if author.startswith("Dr."):
            authors[i] = author.split("Dr.")[1][1:]
        if "Stefan 'Krempl" == author:
             authors[i] = 'Stefan Krempl'
        if 'Wolf von Dewitz (dpa)' == author:
            authors[i] = 'Wolf von Dewitz'
        if 'Veronika Szentpetery' == author or 'Veronika Szentpetery-Kessler' == author:
            authors[i] = 'Veronika Szentpétery-Kessler'
        if 'Übersetzung: Ben Schwan' == author:
            authors[i] = 'Ben Schwan'
        if 'press-inform' == author:
            authors[i] = 'pressinform'
            
        material_dpa = ['Mit Material von dpa', 'mit Material von dpa','mt Material von dpa']
        if author in material_dpa:
             authors[i] = 'mit Material der dpa'
        
        dpa = ['den dpa-Korrespondentinnen', 'dpa-AFX', 'dpa-Korrespondenten']
        if author in dpa:
            authors[i] = 'dpa'          

##  Durchschnittliche Anzahl Autoren pro Artikel

In [ ]:
authors_count_per_article = np.array([len(authors) for authors in authors_article])
np.mean(authors_count_per_article)

In [ ]:
plt.hist(authors_count_per_article)
plt.show()

## Meiste Autoren pro Artikel

In [ ]:
print(np.max(authors_count_per_article))
print(authors_article[np.argmax(authors_count_per_article)])

## Artikel ohne angegebenen Autor

In [ ]:
np.count_nonzero(authors_count_per_article==0)

## Häufigste Autoren

In [ ]:
authors = [author for authors in authors_article for author in authors]
authors_df = pd.DataFrame(authors,columns=['author_name'])

In [ ]:
authors_df = authors_df.groupby(['author_name']).size()
authors_df = pd.DataFrame(authors_df).reset_index()
authors_df = authors_df.rename(columns={0:"number_articles"})
authors_df = authors_df.sort_values(by='number_articles',ascending=False)
authors_df.head(10)

In [ ]:
authors_df.head(20).plot(kind='bar',x = "author_name",y='number_articles',figsize=(10,10))
plt.show()

## Themen der häufigsten Autoren

In [ ]:
author_topics = []
for authors , article in zip(authors_article,articles):
    if "dpa" in authors:
        author_topics += article["related_topics"]

print("dpa")
pd.DataFrame(author_topics).groupby([0]).size().to_frame().rename(columns={0:"topic_frequency"}).sort_values(by="topic_frequency",ascending=False).head(5)

In [ ]:
author_topics = []
for authors , article in zip(authors_article,articles):
    if "Andreas Wilkens" in authors:
        author_topics += article["related_topics"]

print("Andreas Wilkens")
pd.DataFrame(author_topics).groupby([0]).size().to_frame().rename(columns={0:"topic_frequency"}).sort_values(by="topic_frequency",ascending=False).head(5)

In [ ]:
author_topics = []
for authors , article in zip(authors_article,articles):
    if "Stefan Krempl" in authors:
        author_topics += article["related_topics"]
print("Stefan Krempl")        
pd.DataFrame(author_topics).groupby([0]).size().to_frame().rename(columns={0:"topic_frequency"}).sort_values(by="topic_frequency",ascending=False).head(5)

In [ ]:
author_topics = []
for authors , article in zip(authors_article,articles):
    if "Ben Schwan" in authors:
        author_topics += article["related_topics"]
print("Ben Schwan")       
pd.DataFrame(author_topics).groupby([0]).size().to_frame().rename(columns={0:"topic_frequency"}).sort_values(by="topic_frequency",ascending=False).head(5)

## Häufige Autoren-Paare

In [ ]:
author_pairs = []
for authors in authors_article:
    author_pairs += list(itertools.combinations(authors, 2))

In [ ]:
author_pairs_df = pd.DataFrame(np.ones(len(author_pairs)))
author_pairs_df["author_pair"] = author_pairs

In [ ]:
author_pairs_df = author_pairs_df.groupby(["author_pair"]).count()
author_pairs_df = author_pairs_df.reset_index()
author_pairs_df = author_pairs_df.rename(columns={0:"number_author_pair"})
author_pairs_df = author_pairs_df.sort_values(by='number_author_pair',ascending=False)
author_pairs_df.head(10)

# Überschrift
Eine Überschrift ist im Schnitt 7,4 Zeichen lang. Die längste Überschrift umfasst 18 Zeichen. Die häufigsten Wörter als auch Lemmas sind Apple, US (US-amerkanischen (lemma)) und EU.

In [ ]:
headings = [article["heading"] for article in articles]
headings_concat = ' '.join(headings)

In [ ]:
# remove stopwords, and numbers, and anything that's not alphanumeric or underscore
heading_tokens = nltk.word_tokenize(headings_concat)
headings_clean = " ".join([w for w in heading_tokens if not w.lower() in STOP_WORDS])
headings_clean = re.sub(r'[^\w]', ' ', headings_clean)
#headings_clean = re.sub(r'[0-9]+', '', headings_clean)

## Durchschnittliche Länge

In [ ]:
heading_lens = []
for heading in headings:
    heading_len = len(heading.split())
    heading_lens.append(heading_len)
    
print('Durschnittliche Länge der Headings:',np.mean(heading_lens))
print('Längste Heading:',np.max(heading_lens),"\n", headings[np.argmax(heading_lens)])
print('Kürzeste Heading:',np.min(heading_lens),"\n", headings[np.argmin(heading_lens)])
plt.boxplot(heading_lens)
plt.show()

## Häufigste Wörter

In [ ]:
STOPWORDS.update(STOP_WORDS)
wordcloud = WordCloud(background_color="white",width=1920, height=1080, max_words=75).generate(headings_clean)

plt.figure(figsize=(13, 8), dpi=80)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
heading_words_list = headings_clean.split() 
heading_words_df = pd.DataFrame(heading_words_list,columns=['words'])
df = heading_words_df.groupby(['words']).size().to_frame().rename(columns={0:"number_words"})
sorted_df = df.sort_values(by='number_words',ascending=False)
sorted_df.head(15)

In [ ]:
plt.figure(figsize=(25, 15), dpi=100)
sorted_df.head(25).plot.bar()
plt.show()

## Häufigste Lemmas

In [ ]:
headings_clean_spacy = nlp(headings_clean)
headings_clean_lemmas = " ".join([token.lemma_ for token in headings_clean_spacy])

In [ ]:
wordcloud = WordCloud(background_color="white",width=1920, height=1080, max_words=75).generate(headings_clean_lemmas)

plt.figure(figsize=(13, 8), dpi=80)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
heading_words_list = headings_clean_lemmas.split() 
heading_words_df = pd.DataFrame(heading_words_list,columns=['words'])
df = heading_words_df.groupby(['words']).size().to_frame().rename(columns={0:"number_words"})
sorted_df = df.sort_values(by='number_words',ascending=False)
sorted_df.head(15)

In [ ]:
plt.figure(figsize=(25, 15), dpi=100)
sorted_df.head(25).plot.bar()
plt.show()

# Lead

Ein Lead ist im Schnitt 21 Wörter lang. Der längste Lead umfasst 47 Wörter. Die häufigsten Wörter sind ähnlich zu denen der Überschriften, allerdings verdrängen die Wörter Deutschland und Unternehmen, die Wörter Apple und EU von den ersten Plätzen. Die häufigsten Lemmas unterscheiden sich ebenfalls nur gering von den häufigsten Wörtern.

In [ ]:
leads = [article["lead"] for article in articles]
leads_concat = ' '.join(leads)

In [ ]:
# remove stopwords, and numbers, and anything that's not alphanumeric or underscore
leads_tokens = nltk.word_tokenize(leads_concat)
leads_clean = " ".join([w for w in leads_tokens if not w.lower() in STOP_WORDS])
leads_clean = re.sub(r'[^\w]', ' ', leads_clean)
#leads_clean = re.sub(r'[0-9]+', '', leads_clean)

## Durchschnittliche Länge

In [ ]:
lead_lens = []
for lead in leads:
    lead_len = len(lead.split())
    lead_lens.append(lead_len)
    
print('Durschnittliche Länge der Leads:',np.mean(lead_lens))
print('Längster Lead:',np.max(lead_lens))
print('Kürzester Lead:',np.min(lead_lens))
plt.boxplot(lead_lens)
plt.show()

## Häufigste Wörter

In [ ]:
STOPWORDS.update(STOP_WORDS)
wordcloud = WordCloud(background_color="white",width=1920, height=1080, max_words=75).generate(leads_clean)

plt.figure(figsize=(18, 8), dpi=80)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
lead_words_list = leads_clean.split() 
lead_words_df = pd.DataFrame(lead_words_list,columns=['words'])
df = lead_words_df.groupby(['words']).size().to_frame().rename(columns={0:"number_words"})
sorted_df = df.sort_values(by='number_words',ascending=False)
sorted_df.head(15)

In [ ]:
plt.figure(figsize=(25, 15), dpi=100)
sorted_df.head(25).plot.bar()
plt.show()

## Häufigste Lemmas

In [ ]:
nlp.max_length = 1628578

In [ ]:
leads_clean_spacy = nlp(leads_clean)
leads_clean_lemmas = " ".join([token.lemma_ for token in leads_clean_spacy])

In [ ]:
wordcloud = WordCloud(background_color="white",width=1920, height=1080, max_words=75).generate(leads_clean_lemmas)

plt.figure(figsize=(13, 8), dpi=80)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
leads_words_list = leads_clean_lemmas.split() 
leads_words_df = pd.DataFrame(leads_words_list,columns=['words'])
df = leads_words_df.groupby(['words']).size().to_frame().rename(columns={0:"number_words"})
sorted_df = df.sort_values(by='number_words',ascending=False)
sorted_df.head(15)

In [ ]:
plt.figure(figsize=(25, 15), dpi=100)
sorted_df.head(25).plot.bar()
plt.show()

# Text
Texte sind durchschnittlich 350 Zeichen lang. Der längste Text erstreckt sich über 7558 Zeichen. Genau ein Text umfasst 0 Zeichen, da im Artikel span-Tags statt der üblichen p-Tags verwendet wurden. Die häufigsten Wörter sind Prozent, Unternehmen und Euro.

In [ ]:
texts = [article["atricle_text"] for article in articles]
texts_concat = ' '.join(texts)

In [ ]:
# remove stopwords, and numbers, and anything that's not alphanumeric or underscore
texts_tokens = nltk.word_tokenize(texts_concat)
texts_clean = " ".join([w for w in texts_tokens if not w.lower() in STOP_WORDS])
texts_clean = re.sub(r'[^\w]', ' ', texts_clean)
texts_clean = re.sub(r'[0-9]+', '', texts_clean)

## Durchschnittliche Länge

In [ ]:
text_lens = []
for text in texts:
    text_len = len(text.split())
    text_lens.append(text_len)
    
print('Durschnittliche Länge der Texte:',np.mean(text_lens))
print('Längster Text:',np.max(text_lens))
print('Kürzester Text:',np.min(text_lens))
plt.boxplot(text_lens)
plt.show()

## Häufigste Wörter

In [ ]:
STOPWORDS.update(STOP_WORDS)
wordcloud = WordCloud(background_color="white",width=1920, height=1080, max_words=75).generate(texts_clean)

plt.figure(figsize=(13, 8), dpi=80)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
text_words_list = texts_clean.split() 
text_words_df = pd.DataFrame(text_words_list,columns=['words'])
df = text_words_df.groupby(['words']).size().to_frame().rename(columns={0:"number_words"})
sorted_df = df.sort_values(by='number_words',ascending=False)
print(sorted_df.head(15))
plt.figure(figsize=(25, 15), dpi=100)
sorted_df.head(25).plot.bar()
plt.show()

## Häufigste Lemmas

In [ ]:
nlp.max_length = len(texts_clean)+1

In [ ]:
texts_clean_spacy = nlp(texts_clean)
texts_clean_lemmas = " ".join([token.lemma_ for token in texts_clean_spacy])

In [ ]:
STOPWORDS.update(STOP_WORDS)
wordcloud = WordCloud(background_color="white",width=1920, height=1080, max_words=75).generate(texts_clean_lemmas)

plt.figure(figsize=(13, 80), dpi=80)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
text_words_list = texts_clean_lemmas.split() 
text_words_df = pd.DataFrame(text_words_list,columns=['words'])
df = text_words_df.groupby(['words']).size().to_frame().rename(columns={0:"number_words"})
sorted_df = df.sort_values(by='number_words',ascending=False)
print(sorted_df.head(15))
plt.figure(figsize=(25, 15), dpi=100)
sorted_df.head(25).plot.bar()
plt.show()